# Fake News Classification

In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('news.csv')

In [3]:
df.shape

(6335, 4)

In [4]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
# Since text column is more important to us than other column
df = df[['text', 'label']]

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.label = le.fit_transform(df.label)
df.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,U.S. Secretary of State John F. Kerry said Mon...,1
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,It's primary day in New York and front-runners...,1


In [7]:
# cleaning text 
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stp_words = stopwords.words('english')
from nltk.stem import PorterStemmer
regex = re.compile('[^a-zA-Z]')
pst = PorterStemmer()
li = []
for sentence in df.text:
    tokens = word_tokenize(sentence)
    temp_li = []
    for word in tokens:
        if (word not in stp_words) & (len(word) > 2):
            word = pst.stem(word)
            word = regex.sub('',word)
            temp_li.append(word)
    temp_li = ' '.join(temp_li)
    li.append(temp_li)
li[0]

'daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi the word unpreced thrown around often elect ought retir but still unpreced nomine major polit parti war fbi but exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect the fbi attack everyon obama cnn hillari peopl circul letter attack comey there current media hit piec lambast target trump surpris clinton alli start run attack ad fbi the fbi leadership warn entir leftw establish form lynch mob continu go hillari and fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton the covert struggl fbi agent obama doj peopl gone explos public the new york time compar comey edgar hoover it bizarr headlin jame comey role recal hoover fbi fairli not practic admit front spout nonsens the boston globe publish column call comey resign not outdo

## Applying TF-IDF

In [8]:
# converting the text to vector
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words = 'english', max_df = 0.7)
x = vect.fit_transform(li)
x = pd.DataFrame(x.toarray(), columns = vect.get_feature_names())

In [9]:
y = df['label']

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

## Applying Random Forest

In [11]:

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
pred = model.predict(x_test)
print('accuracy_score = ',accuracy_score(y_test, pred))
print('confusion_matrix = ', confusion_matrix(y_test, pred))

C:\Users\uru\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy_score =  0.8396464646464646
confusion_matrix =  [[713  78]
 [176 617]]


## Applying Passive Aggrasive Algorithm

In [12]:
from sklearn.linear_model import PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter = 50)
pac.fit(x_train, y_train)
pred = pac.predict(x_test)
print('accuracy_score = ',accuracy_score(y_test, pred))
print('confusion_matrix = ', confusion_matrix(y_test, pred))

C:\Users\uru\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


accuracy_score =  0.9349747474747475
confusion_matrix =  [[750  41]
 [ 62 731]]
